In [1]:
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import sklearn.linear_model  as sk  # namespace conflict with pyspark
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import scale
from sklearn.model_selection import KFold , cross_val_score
from sklearn.preprocessing import StandardScaler
import pandas as pd
from helper import *
import tensorflow as tf

pd.options.display.max_columns = None
import findspark
import pyspark
from pyspark.sql import SQLContext
from pyspark.ml.regression import LinearRegression
from pyspark.ml.feature import VectorAssembler
from pyspark.mllib.regression import LabeledPoint, LinearRegressionWithSGD
from pyspark.sql.functions import col

from pyspark.ml.linalg import VectorUDT
from pyspark.sql.functions import udf
from pyspark.ml.evaluation import RegressionEvaluator



# Stuff that needs to be done

- Find another framework.
    - Sklearn
    - Spark
    - Tensorflow/R
- Algorithms
    - Linear Regression
    - Logistic Regression
    - Maybe SVM /Random forest
- Find two metrics 
    - RMSE 
    - Some other
- Do 70/30 split training and 10 cross validation
    - only did cross validation
- Find another dataset 
    - Sum data with noise 
    -  Maybe housing data set
    

## Import DataSet

In [2]:
sumdata_url = "https://www.dropbox.com/sh/euppz607r6gsen2/AABABUTdx7YqCeBquA1Ky7z8a/The%20SUM%20dataset?dl=1#"
housing_price_url = "https://www.dropbox.com/sh/euppz607r6gsen2/AAAVLZzU4E7ro0BiRzPG3pP8a/House%20Prices?dl=1"
all_urls = [sumdata_url, housing_price_url]

In [3]:
get_data(all_urls) # retrieves the data if there is no data folder

In [59]:
sumdata_noise_path = "data/with noise/The SUM dataset, with noise.csv"
sumdata_path = "data/without noise/The SUM dataset, without noise.csv"
housing_price_path = "data/housing dataset.csv" # has more than 30 features
# need one more
# what a brilliant idea to name files with space

In [60]:
data_chunks = [100, 500, 1000, 5000, 10000, 50000, 100000, 500000,
1000000, 5000000, 10000000, 50000000, 100000000]

## Load datasets sum_noise

In [61]:
sumdata_noise = pd.read_csv(sumdata_noise_path, delimiter=";")
sumdata_noise.head(n=1)

,Instance,Feature 1,Feature 2,Feature 3,Feature 4,Feature 5 (meaningless but please still use it),Feature 6,Feature 7,Feature 8,Feature 9,Feature 10,Noisy Target,Noisy Target Class
0,1,62485,58472,84200,86181,75529,136939,150633,230058,246491,257336,1352179,Very Large Number


## Preprocess sum_noise dataset

- Remove 'Instance' as it simply represents the row number
- Extract 'Nosiy Target' as regression target
- Extract 'Nosiy Class' as classification target
- Extract rest columns as explananatory variables
- Apply Feature Scaling to the dataset 

- Ensure all dataframe has been converted to numpy array


### Create same base line for Spark, Keras, Sklearn implementation
- Using the same feature scaled data for all implementation
- Same number iterations
- Same cost function for minimising(for gradient descent)

In [62]:
# Remove 'Instance' as it simply represents the row number



sumdata_noise.drop('Instance', axis = 1)

# Extract 'Nosiy Target' as regression target
sumdata_noise_reg_Y = sumdata_noise.loc[:, ['Noisy Target']]
# Extract 'Nosiy Target Class' as regression target
sumdata_noise_classif_Y = sumdata_noise.loc[:, ['Noisy Target Class']]

# Extract rest columns as explananatory variables
sumdata_noise_X =sumdata_noise.drop(['Instance','Noisy Target Class'
                                    ], axis = 1)


### Feature scaling
- Keeping them as dataframes (transform returns ndarray)

In [63]:
scX = StandardScaler()
scY = StandardScaler()
columns = sumdata_noise_X.columns

sumdata_noise_X[columns] = scX.fit_transform(sumdata_noise_X)
columns = sumdata_noise_reg_Y.columns
sumdata_noise_reg_Y[columns] = scX.fit_transform(sumdata_noise_reg_Y)


sumdata_noise_scaled = pd.concat([sumdata_noise_X,sumdata_noise_reg_Y])

### Linear Regression using pyspark

In [64]:
sc = pyspark.SparkContext()
sqlContext = SQLContext(sc)


ValueError: Cannot run multiple SparkContexts at once; existing SparkContext(app=pyspark-shell, master=local[*]) created by __init__ at <ipython-input-9-8030b2fc72d9>:1 

In [65]:
from  pyspark.ml.tuning import CrossValidator, ParamGridBuilder

In [66]:
def diff_sq(n):
    return (n[0] - n[1])**2

In [67]:
def sum_1(n,n2):
    return (n + n2)

In [68]:
from math import sqrt
def spark_linear_regression(data_df):
    """
        This doesnt have cross validation 
        doesnt have test split
    """
    input_features = list(sumdata_noise.columns)[:-2] 
    
    data = data_df.select(input_features)\
      .rdd\
      .map(lambda line:LabeledPoint(line[-1],line[1:-1]))
 
    
    lr = LinearRegressionWithSGD( )

    # Fit 2 models, using different regularization parameters
    model = lr.train(data,iterations=1000)
    
    # Evaluate the model on training data
    valuesAndPreds =data.map(lambda p: (p.label, model.predict(p.features))) 
     
    MSE = valuesAndPreds\
        .map(diff_sq)\
        .reduce(sum_1) / valuesAndPreds.count()
    print("Mean Squared Error = " + str(sqrt(MSE)))

    

In [69]:
for chunk in data_chunks:

    if chunk > sumdata_noise.shape[0]: # if chunk is greater than the no. of examples
        break
        
        
    data = sumdata_noise[:chunk]
    data = data.drop('Noisy Target Class', axis=1)
    columns = data.columns
    
    data[columns] = StandardScaler().fit_transform(data)
    data_df = sqlContext.createDataFrame(data)
     
    
    spark_linear_regression(data_df)
    print("\n")

/home/jibin/anaconda3/lib/python3.6/site-packages/pyspark/mllib/regression.py:281: UserWarning: Deprecated in 2.0.0. Use ml.regression.LinearRegression.
  warnings.warn("Deprecated in 2.0.0. Use ml.regression.LinearRegression.")


Mean Squared Error = 0.2141731306672613


Mean Squared Error = 0.2336466274817069


Mean Squared Error = 0.24004858367273027


Mean Squared Error = 0.242831777382388


Mean Squared Error = 0.24331256292359593


Mean Squared Error = 0.24238415215080497


Mean Squared Error = 0.242741461062573


Mean Squared Error = 0.244050615169765




In [70]:
def sklearn_linear_regression_sgd( X_train, X_test, y_train, y_test): 
    """
        This might not be much of use as I wasn't able to control 
        the learning rate, number of iterations
    """
    clf = sk.SGDRegressor()
    model = clf.fit(X_train, y_train.ravel())
    preds = model.predict(X_test)
    return mean_squared_error(y_test, preds)**0.5

In [71]:
def model_test_cross( data_x, data_y, dataset_name): 
    
    errors = np.zeros((3, 10)) # 
    for chunk in data_chunks:

        if chunk > sumdata_noise.shape[0]: # if chunk is greater than the no. of examples
            break

        X = data_x.head(n = chunk)
        y = data_y.head(n = chunk)
        
        X = X.as_matrix()
        y = y.as_matrix() 
        kf = KFold(n_splits=10)
        
        
        for (i, (train_index, test_index)) in enumerate(kf.split(X)):
            
            
            X_train, X_test = X[train_index], X[test_index]
            y_train, y_test = y[train_index], y[test_index]
            # sklearn storing 
            errors[0][i] = sklearn_linear_regression_sgd( X_train, X_test, y_train, y_test)
            #
            #errors[0][1] = sklearn_linear_regression_sgd( X_train, X_test, y_train, y_test)
        
        # calculates mean across rows
        mean_error = np.mean(errors, axis=0) 
        print ("Data set: {} Chunk Size: {} Error: {}".format(dataset_name,chunk, mean_error[0]))

In [72]:
model_test_cross( sumdata_noise_X, sumdata_noise_reg_Y, "Sum data with noise")

Data set: Sum data with noise Chunk Size: 100 Error: 0.02240438583606064
Data set: Sum data with noise Chunk Size: 500 Error: 0.034185244683955514
Data set: Sum data with noise Chunk Size: 1000 Error: 0.03361810209604333
Data set: Sum data with noise Chunk Size: 5000 Error: 0.02639941556864127
Data set: Sum data with noise Chunk Size: 10000 Error: 0.020960751781316834
Data set: Sum data with noise Chunk Size: 50000 Error: 0.00636657752448506
Data set: Sum data with noise Chunk Size: 100000 Error: 0.002108051672184291
Data set: Sum data with noise Chunk Size: 500000 Error: 0.0002771586887816033


In [49]:
n = np.array([[1,3], [5,6]])

In [51]:
r =np.mean(n
    , axis=0) 

In [52]:
r[0]

3.0